# Fetching crypto and tweets data.

In [1]:
from sklearn.preprocessing import OrdinalEncoder
from crypto_api import CryptoApi
import pandas as pd
import numpy as np
import re


btc_df = pd.read_csv(r'Data\btc_data.csv',usecols=lambda x: x != "Unnamed: 0")

# Analysis of tweets

In [2]:
'''
TODO: GENERAL DATA TASKS:
0) find a way to deal with multiple tweets for a day
1) merge 2 datasets into 
2) imput missing data, maybe try interpolation or expectation maximization
    2.1) compare with mean, median imput methods
3) ivestigate relationship within data, maybe correlation matrix etc
'''

tweets_df = pd.read_csv(r'Data/elon_tweets.csv', index_col=0)
tweets_df['date'] = pd.to_datetime(tweets_df['date'])
                                                
tweets_df = (tweets_df
             .dropna(axis=1, how='all')
             .drop(['vibe','cashtags'], axis=1)) # 1 and 18 notna values respectively  

## Dealing with sparse columns

In [3]:
sparse_cols = tweets_df.columns[tweets_df.notnull().mean() < 1.0].values.copy()

mod_tweets_df = tweets_df.copy()
mod_tweets_df = (mod_tweets_df[mod_tweets_df['lang']=='en']
                 .drop(['id','url','source','sourceUrl'], axis=1)                 
                 .reset_index(drop=True)
                 .copy())

mod_tweets_df = mod_tweets_df.drop(['lang'], axis=1)

## Data cleaning and preprocessing

In [4]:
encoder = OrdinalEncoder()
mod_tweets_df['sourceLabel_encoded'] = encoder.fit_transform(mod_tweets_df['sourceLabel'].values.reshape(-1, 1))
mod_tweets_df['isReplied']   = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['inReplyToUser']]
mod_tweets_df['isMentioned'] = [0 if type(tweet)==float else 1 for tweet in mod_tweets_df['mentionedUsers']]

#mod_tweets_df = mod_tweets_df.drop(['sourceLabel','inReplyToUser','mentionedUsers'], axis=1)


def extract_dict(line: str, prepare_to_df: False):
    """Extracts data from a dict represented as string and makes it a dict.

    ## Parameters:
        line (str): row of a Series/DataFrame to be preprocessed.
        prepare_to_df (bool): prepares extracted dict to be wrapped into DataFrame.

    ## Returns:
        dict: extracted dict from string.
    """    

    extracted_content = dict(re.findall(r"'(\w+)': '?({.*}|datetime.datetime\(.*\)|[\w\d/:\. ]*)'?", line))
    
    # Wraps dict values into lists to be easily represented as a DataFrame row.
    if prepare_to_df:
        for key,value in extracted_content.items():
            if value == '':
                extracted_content[key] = [None]
            else:
                extracted_content[key] = value
        
    return extracted_content


new_df = mod_tweets_df.copy()     
extracted_df = (pd.DataFrame([*mod_tweets_df['user']
                              .apply(lambda x: extract_dict(x, True))])
                )

new_df = (pd.concat([new_df, extracted_df], axis=1)
            .drop(['user','username','id','displayname','verified','created',
                    'location','protected','profileImageUrl','profileBannerUrl',
                    'rawDescription','renderedDescription','favouritesCount',
                    'friendsCount','mediaCount','statusesCount'], axis=1))

In [5]:
# Converting columns containing numbers to int after extraction.
for column in new_df:
    if 'Count' in column:
        new_df[column] = new_df[column].astype('Int64').copy()

In [6]:
new_df[['rawContent','isReplied','isMentioned']].query("rawContent.str.contains('@')")

new_df['mentionsCount'] = new_df['rawContent'].str.count(r'@[\w\d]+')

In [7]:
new_df['mentions'] = new_df['rawContent'].apply(lambda x : re.findall(r'(@[^\s]+)', x))

count = 0
for a,b in new_df[['mentionsCount','mentions']].values:
    if a==len(b):
        count +=1 
print(count==len(new_df))

True


In [8]:
new_df['charCount'] = new_df['rawContent'].apply(lambda x: len(x))
new_df = new_df.drop('descriptionLinks', axis=1)

In [9]:
#links = tweets_df[tweets_df.columns[tweets_df.columns.isin(new_df.columns)==False]]['links'].value_counts().copy()

tweets_df[tweets_df['rawContent']=='True']['quotedTweet'].iloc[0]

'{\'url\': \'https://twitter.com/ggreenwald/status/1625871270737809408\', \'date\': datetime.datetime(2023, 2, 15, 14, 54, 52, tzinfo=datetime.timezone.utc), \'rawContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'renderedContent\': "The corporate media\'s ability to -- overnight -- turn anyone who dissents in anyway into some sort of fascist or even Hitler-like figure, and then have millions of their followers go around mindlessly repeating it, is both impressive and chilling:", \'id\': 1625871270737809408, \'user\': {\'username\': \'ggreenwald\', \'id\': 16076032, \'displayname\': \'Glenn Greenwald\', \'rawDescription\': \'Journalist; Author; Host, @SystemUpdate_; Columnist, @Folha; Co-Founder: The Intercept, @TheInterceptBr; @abrigo_hope, @FreedomofPress, @ongcriadefavela.

# Model Bulding

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, utils, models
from gensim.models import ldamodel
import spacy


def lemmatization(nlp, texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for text in texts:
        doc = nlp(text)
        cleaned_text = []
        for token in doc:
            if token.ent_type_ != 'URL' and not token.is_stop and token.pos_ in allowed_postags:
                cleaned_text.append(token.lemma_)
        final = ' '.join(cleaned_text)
        texts_out.append(final)

    return texts_out

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
url_pattern = [{"label": "URL",
                "pattern": [{"LIKE_URL": True}]}]

ruler = nlp.add_pipe('entity_ruler', before='ner')
ruler.add_patterns(url_pattern)

lemmatized_texts = lemmatization(nlp, new_df['rawContent'])

In [112]:
def gen_words(texts):
    final = []
    for text in texts:
        new = utils.simple_preprocess(text)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

In [113]:
#bi-tri grams

bigrams_phrases  = models.Phrases(data_words, min_count=3, threshold=50)
trigrams_phrases = models.Phrases(bigrams_phrases[data_words], threshold=50)

bigram  = models.phrases.Phraser(bigrams_phrases)
trigram = models.phrases.Phraser(trigrams_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigram(texts):
    return [trigram[bigram[doc]] for doc in texts]

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigram(data_bigrams)

In [114]:
id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = models.TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id,_ in tfidf[bow]]
    bow_idf = [id for id,_ in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    
    for item in drops:
        words.append(id2word[item])
    
    # words with tfidf score of 0 will be missing
    words_missing_in_tfidf = [id for id in bow_idf if id not in tfidf_ids]
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [115]:
# id2word = corpora.Dictionary(data_words)
# corpus = [id2word.doc2bow(text) for text in data_words]

In [131]:
lda_model = ldamodel.LdaModel(corpus,  num_topics=35, 
                              id2word=id2word, random_state=1, 
                              update_every=1, chunksize=3000, 
                              passes=10, alpha='auto')

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

d:\System\Python\lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.397900 -0.170909       1        1  8.070255
11    -0.426327  0.051671       2        1  6.889466
14    -0.076257 -0.179389       3        1  6.775016
17    -0.122891 -0.389517       4        1  5.484142
18     0.071245  0.409259       5        1  5.381564
1     -0.125089  0.142412       6        1  5.380448
4     -0.329680  0.216808       7        1  5.165948
19    -0.263376 -0.284229       8        1  5.034979
16     0.058297 -0.423624       9        1  4.936140
10     0.277957 -0.334343      10        1  4.878659
9      0.125895 -0.227006      11        1  4.582080
5      0.208509  0.327761      12        1  4.411040
3     -0.253807 -0.018936      13        1  4.392463
7      0.110558  0.083234      14        1  4.370954
12    -0.212443  0.358578      15        1  4.347237
8     -0.049837  0.341538      16        1  4.254281
2      0.274259 -0.135722      17        1  4.208156
15     0.406502 -0.076488      18        1  3.897591
6      0.390309  0.085439      19        1  3.855578
13     0.334075  0.223463      20        1  3.684005, topic_info=            Term         Freq        Total Category  logprob  loglift
70          good  1115.000000  1115.000000  Default  30.0000  30.0000
32         great   759.000000   759.000000  Default  29.0000  29.0000
29          true   673.000000   673.000000  Default  28.0000  28.0000
230         time   797.000000   797.000000  Default  27.0000  27.0000
287         year   498.000000   498.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
1436       floor    26.754226    27.912008  Topic20  -4.9672   3.2588
2523     trigger    25.240553    26.398335  Topic20  -5.0255   3.2563
677   simulation    23.945083    25.102865  Topic20  -5.0782   3.2540
402         view    61.297557   101.276123  Topic20  -4.1382   2.7991
1225        city    43.870093    80.869753  Topic20  -4.4727   2.6896

[818 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1353     11  0.986948        able
186       5  0.993015  absolutely
911      14  0.973884  accelerate
3039      4  0.973012      accord
3032     10  0.983130     account
...     ...       ...         ...
1892     18  0.980808      writer
1568     17  0.992496       wrong
287       9  0.996692        year
169      12  0.992240    year_ago
639      14  0.961658   yesterday

[1041 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 12, 15, 18, 19, 2, 5, 20, 17, 11, 10, 6, 4, 8, 13, 9, 3, 16, 7, 14])